In [3]:
from google.colab import files

In [4]:
files.upload()

Saving drug_names.tsv to drug_names.tsv


{'drug_names.tsv': b"CID100000085\tcarnitine\nCID100000119\tgamma-aminobutyric\nCID100000137\t5-aminolevulinic\nCID100000143\tleucovorin\nCID100000146\t5-methyltetrahydrofolate\nCID100000158\tPGE2\nCID100000159\tprostacyclin\nCID100000160\tprostaglandin\nCID100000175\tacetate\nCID100000187\tacetylcholine\nCID100000191\tadenosine\nCID100000206\tglucose\nCID100000214\tPGE1\nCID100000222\tammonia\nCID100000232\targinine\nCID100000244\tbenzyl\nCID100000247\tbetaine\nCID100000271\tcalcium\nCID100000297\tgraphene\nCID100000298\tchloramphenicol\nCID100000303\tbile\nCID100000305\tcholine\nCID100000311\tcitric\nCID100000312\tchloride\nCID100000338\tsalicylate\nCID100000401\tD-cycloserine\nCID100000444\tbupropion\nCID100000450\testradiol\nCID100000453\tmannitol\nCID100000564\tEACA\nCID100000581\tN-acetylcysteine\nCID100000596\tcytarabine\nCID100000598\tmesna\nCID100000612\tlactate\nCID100000679\tDMSO\nCID100000681\tdopamine\nCID100000698\testrone\nCID100000699\tOestrogen\nCID100000700\tmonoethan

In [5]:
import pandas as pd

# Load drug names
drug_names = pd.read_csv("drug_names.tsv", sep="\t", header=None, names=["STITCH_ID", "Drug_Name"])
print("Drug Names:\n", drug_names.head())

Drug Names:
       STITCH_ID                 Drug_Name
0  CID100000085                 carnitine
1  CID100000119        gamma-aminobutyric
2  CID100000137          5-aminolevulinic
3  CID100000143                leucovorin
4  CID100000146  5-methyltetrahydrofolate


In [6]:
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [7]:
# Load drug-side effect links
side_effects = pd.read_csv("meddra_all_se.tsv", sep="\t", header=None,
                            names=["STITCH_ID", "UMLS_ID", "MedDRA_PT", "MedDRA_LL", "SideEffect"])
print("Side Effects:\n", side_effects.head())

Side Effects:
                  STITCH_ID   UMLS_ID MedDRA_PT MedDRA_LL  \
CID100000085  CID000010917  C0000729       LLT  C0000729   
CID100000085  CID000010917  C0000729        PT  C0000737   
CID100000085  CID000010917  C0000737       LLT  C0000737   
CID100000085  CID000010917  C0000737        PT  C0687713   
CID100000085  CID000010917  C0000737        PT  C0000737   

                         SideEffect  
CID100000085       Abdominal cramps  
CID100000085         Abdominal pain  
CID100000085         Abdominal pain  
CID100000085  Gastrointestinal pain  
CID100000085         Abdominal pain  


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [11]:
# Use only top 100 drugs and side effects for simplicity
top_drugs = side_effects['STITCH_ID'].value_counts().head(100).index
filtered = side_effects[side_effects['STITCH_ID'].isin(top_drugs)]

# Encode drugs and side effects as integers
drug_encoder = LabelEncoder()
se_encoder = LabelEncoder()

filtered['drug_encoded'] = drug_encoder.fit_transform(filtered['STITCH_ID'])
filtered['se_encoded'] = se_encoder.fit_transform(filtered['SideEffect'])

<ipython-input-11-ce6c336f1687>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['drug_encoded'] = drug_encoder.fit_transform(filtered['STITCH_ID'])
<ipython-input-11-ce6c336f1687>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['se_encoded'] = se_encoder.fit_transform(filtered['SideEffect'])


In [12]:
# Define input features (just drug ID) and target (side effect)
X = filtered[['drug_encoded']]
y = filtered['se_encoded']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [13]:
# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.00044972695149373597
Report:
               precision    recall  f1-score   support

           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         6
          12       0.00      0.00      0.00         2
          13       0.00      0.00      0.00        23
          15       0.00      0.00      0.00        33
          16       0.00      0.00      0.00         1
          19       0.00      0.00      0.00        44
          21       0.00      0.00      0.00         7
          22       0.00      0.00      0.00        24
          23       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         4
          27       0.00      0.00      0.00         7
          30       0.00      0.00      0.00        28
          31       0.00      0.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_